# Задача

Реализовать класс `MyBinaryLogisticRegression` для работы с логистической регрессией. Обеспечить возможность использования `l1`, `l2` и `l1l2` регуляризации и реализовать слудующие методы решения оптимизационной задачи:

*   Градиентный спуск
*   Стохастический градиентный спуск
*   Метод Ньютона

Обосновать применимость/не применимость того или иного метода оптимизации в случае использованного типа регуляризации.



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler

In [48]:
class MyBinaryLogisticRegression:
    """
    Класс для бинарной логистической регрессии с поддержкой L1, L2 и L1+L2 регуляризаций
    и тремя методами оптимизации: градиентный спуск (GD), стохастический градиентный спуск (SGD)
    и метод Ньютона.

    Parameters:

    lr : float, default=0.1
        Коэффициент обучения (learning rate)

    n_iter : int, default=1000
        Максимальное количество итераций оптимизации.

    tol : float, default=1e-6
        Критерий сходимости: норма градиента меньше tol останавливает оптимизацию.

    penalty : {'l1', 'l2', 'l1l2', None}, default=None
        Тип регуляризации:
        - 'l1'   : L1 регуляризация (Lasso)
        - 'l2'   : L2 регуляризация (Ridge)
        - 'l1l2' : комбинация L1 и L2
        - None   : без регуляризации

    alpha : float, default=0.5
        Для L1L2 регуляризации, доля L1 в комбинированной регуляризации.
        L2 доля = 1 - alpha

    method : {'gd', 'sgd', 'newton'}, default='gd'
        Метод оптимизации:
        - 'gd'     : градиентный спуск
        - 'sgd'    : стохастический градиентный спуск
        - 'newton' : метод Ньютона (только для L2 и L1L2)

    batch_size : int, default=32
        Размер батча для SGD
    """
    def __init__(self, lr=0.1, n_iter=1000, tol=1e-6, penalty=None, alpha=0.5, method='gd', batch_size=32):
        self.lr = lr
        self.n_iter = n_iter
        self.tol = tol
        self.penalty = penalty
        self.alpha = alpha
        self.method = method
        self.batch_size = batch_size
        self.coefs_ = None
        self.intercept_ = None
        self.feature_names_in_ = None

    def sigmoid(self, z):
        """
        Сигмоидная функция
        """
        return 1 / (1 + np.exp(-z))

    def _loss(self, X, y):
        """ 
        Функция потерь (логистическое правдоподобие) с регуляризацией.
        """
        y_pred = self.sigmoid(X @ self.coefs_)
        eps = 1e-15
        loss = -np.mean(y * np.log(y_pred + eps) + (1 - y) * np.log(1 - y_pred + eps))
        if self.penalty == 'l2':
            loss += self.lr * 0.5 * np.sum(self.coefs_ ** 2)
        elif self.penalty == 'l1':
            loss += self.lr * np.sum(np.abs(self.coefs_))
        elif self.penalty == 'l1l2':
            loss += self.lr * (self.alpha * np.sum(np.abs(self.coefs_)) + (1 - self.alpha) * 0.5 * np.sum(self.coefs_ ** 2))
        return loss

    def _gradient(self, X, y):
        """
        Градиент функции потерь с регуляризацией
        """
        y_pred = self.sigmoid(X @ self.coefs_)
        grad = X.T @ (y_pred - y) / X.shape[0]
        if self.penalty == 'l2':
            grad += self.lr * self.coefs_
        elif self.penalty == 'l1':
            grad += self.lr * np.sign(self.coefs_)
        elif self.penalty == 'l1l2':
            grad += self.lr * (self.alpha * np.sign(self.coefs_) + (1 - self.alpha) * self.coefs_)
        return grad

    def _hessian(self, X):
        """
        Гессиан функции потерь (только для L2 и L1L2)
        """
        y_pred = self.sigmoid(X @ self.coefs_)
        W = np.diag(y_pred * (1 - y_pred))
        H = X.T @ W @ X / X.shape[0]
        if self.penalty == 'l2':
            H += self.lr * np.eye(X.shape[1])
        elif self.penalty == 'l1':
            raise ValueError("Hessian не определён для L1 регуляризации")
        elif self.penalty == 'l1l2':
            H += self.lr * (1 - self.alpha) * np.eye(X.shape[1])
        return H

    def fit(self, X: pd.DataFrame, y: pd.Series):
        """
        Обучение модели на данных X и метках y
        """
        X = np.array(X)
        y = np.array(y)
        n_samples, n_features = X.shape
        self.coefs_ = np.zeros(n_features)

        for i in range(self.n_iter):
            if self.method == 'gd':
                grad = self._gradient(X, y)
                self.coefs_ -= self.lr * grad

            elif self.method == 'sgd':
                idx = np.random.choice(n_samples, self.batch_size, replace=False)
                grad = self._gradient(X[idx], y[idx])
                self.coefs_ -= self.lr * grad

            elif self.method == 'newton':
                if self.penalty == 'l1' or self.penalty == 'l1l2':
                    raise ValueError("Метод Ньютона нельзя с L1/L1L2")
                grad = self._gradient(X, y)
                H = self._hessian(X)
                self.coefs_ -= np.linalg.inv(H) @ grad

            if np.linalg.norm(grad) < self.tol:
                break

    def predict_proba(self, X):
        """
        Вероятности положительного класса для данных X
        """
        X = np.array(X)
        return self.sigmoid(X @ self.coefs_)

    def predict(self, X):
        """
        Бинарные предсказания (0/1) для данных X.
        """
        return (self.predict_proba(X) >= 0.5).astype(int)

    def score(self, X, y):
        """
        F1-score для данных X и меток y
        """
        return f1_score(y, self.predict(X))

Продемонстрировать применение реализованного класса на датасете про пингвинов (целевая переменная — вид пингвина). Рассмотреть все возможные варианты (регуляризация/оптимизация). Для категориального признака `island` реализовать самостоятельно преобразование `Target Encoder`, сравнить результаты классификации с `one-hot`. В качестве метрики использовать `f1-score`.

In [24]:
# Считываем данные
df = pd.read_csv('./penguins_binary_classification.csv')
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,year
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,2007
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,2007
3,Adelie,Torgersen,36.7,19.3,193.0,3450.0,2007
4,Adelie,Torgersen,39.3,20.6,190.0,3650.0,2007


In [15]:
df.shape

(274, 7)

In [25]:
# Вид пингвина целевая переменная
df.species.unique()

array(['Adelie', 'Gentoo'], dtype=object)

In [26]:
# Бинарный таргет
df['y'] = (df['species'] == 'Gentoo').astype(int)

In [30]:
# One-hot encoding для island
island_ohe = pd.get_dummies(df['island'], drop_first=True)
df_ohe = pd.concat([df, island_ohe], axis=1)
X_ohe = df_ohe.drop(['island', 'y', 'species'], axis=1)
X_ohe.head()

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,year,island_te,Dream,Torgersen
0,39.1,18.7,181.0,3750.0,2007,0.0,False,True
1,39.5,17.4,186.0,3800.0,2007,0.0,False,True
2,40.3,18.0,195.0,3250.0,2007,0.0,False,True
3,36.7,19.3,193.0,3450.0,2007,0.0,False,True
4,39.3,20.6,190.0,3650.0,2007,0.0,False,True


In [31]:
# Target Encoding
target_mean = df.groupby('island')['y'].mean().to_dict()
df['island_te'] = df['island'].map(target_mean)
X_te = df.copy()
X_te = X_te.drop(['island', 'y', 'species'], axis=1)
X_te.head()

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,year,island_te
0,39.1,18.7,181.0,3750.0,2007,0.0
1,39.5,17.4,186.0,3800.0,2007,0.0
2,40.3,18.0,195.0,3250.0,2007,0.0
3,36.7,19.3,193.0,3450.0,2007,0.0
4,39.3,20.6,190.0,3650.0,2007,0.0


In [32]:
y = df['y']

In [33]:
# стандартизируем
scaler_te = StandardScaler().fit(X_te)
scaler_ohe = StandardScaler().fit(X_ohe)

X_te_scaled = scaler_te.transform(X_te)
X_ohe_scaled = scaler_ohe.transform(X_ohe)

In [34]:
# Разбиваем на тренировочную и тестовую выборку
X_train_te, X_test_te, y_train, y_test = train_test_split(X_te_scaled, y, test_size=0.2, stratify=y, random_state=42)
X_train_ohe, X_test_ohe = train_test_split(X_ohe_scaled, test_size=0.2, stratify=y, random_state=42)

In [36]:
# Эксперименты
regs = ['none', 'l1', 'l2', 'l1l2']
optims = ['gd', 'sgd', 'newton']
results = []

In [41]:
def comp_model(X_tr, X_te, y_tr, y_te, penalty, method):
    model = MyBinaryLogisticRegression(lr=0.1, n_iter=5000, penalty=penalty, method=method)
    model.fit(pd.DataFrame(X_tr), y_tr)
    return model.score(pd.DataFrame(X_te), y_te)

In [49]:
results = []
for feat_name, (X_tr, X_te) in [('Target Encoding', (X_train_te, X_test_te)), ('One-Hot', (X_train_ohe, X_test_ohe))]:
    for penalty in [None,'l2','l1','l1l2']:
        for method in ['gd','sgd','newton']:
            try:
                score = comp_model(X_tr, X_te, y_train, y_test, penalty, method)
                results.append((feat_name, penalty, method, score))
            except Exception as e:
                results.append((feat_name, penalty, method, str(e)))


In [50]:
# Итоговая таблица
print("\nТаблица результатов")
results_df = pd.DataFrame(results, columns=['Encoding','Penalty','Method','F1'])
print(results_df)


Таблица результатов
           Encoding Penalty  Method                              F1
0   Target Encoding    None      gd                             1.0
1   Target Encoding    None     sgd                             1.0
2   Target Encoding    None  newton                             1.0
3   Target Encoding      l2      gd                             1.0
4   Target Encoding      l2     sgd                             1.0
5   Target Encoding      l2  newton                             1.0
6   Target Encoding      l1      gd                             1.0
7   Target Encoding      l1     sgd                             1.0
8   Target Encoding      l1  newton  Метод Ньютона нельзя с L1/L1L2
9   Target Encoding    l1l2      gd                             1.0
10  Target Encoding    l1l2     sgd                             1.0
11  Target Encoding    l1l2  newton  Метод Ньютона нельзя с L1/L1L2
12          One-Hot    None      gd                             1.0
13          One-Hot    None

В рамках лабораторной работы реализован класс бинарной логистической регрессии с поддержкой L1, L2, L1L2 регуляризации и без неё, а также трёх методов оптимизации: градиентного спуска, стохастического градиентного спуска и метода Ньютона. Для категориального признака `island` реализованы One-Hot кодирование и Target Encoding со сглаживанием.

На результирующей таблице видно, что почти все комбинации дают `f1 = 1.0` и для Target Encoding, и для One-Hot, что указывает на высокую линейную разделимость классов. 
Регуляризация практически не влияла на качество при градиентных методах. Метод Ньютона для L1 и L1L2 запрещён, так как функция потерь с L1 не дважды дифференцируема. Градиентный и стохастический спуск применимы ко всем типам регуляризации.

# Теоретическая часть

Пусть данные имеют вид
$$
(x_i, y_i), \quad y_i \in \{1, \ldots,M\}, \quad i \in \{1, \ldots, N\},
$$
причем первая координата набора признаков каждого объекта равна $1$.
Используя `softmax`-подход, дискриминативная модель имеет следующий вид
$$
\mathbb P(C_k|x) = \frac{\exp(\omega_k^Tx)}{\sum_i \exp(\omega_i^Tx)}.
$$
Для написания правдоподобия удобно провести `one-hot` кодирование меток класса, сопоставив каждому объекту $x_i$ вектор $\widehat y_i = (y_{11}, \ldots, y_{1M})$ длины $M$, состоящий из нулей и ровно одной единицы ($y_{iy_i} = 1$), отвечающей соответствующему классу. В этом случае правдоподобие имеет вид
$$
\mathbb P(D|\omega) = \prod_{i = 1}^{N}\prod_{j = 1}^M \mathbb P(C_j|x_i)^{y_{ij}}.
$$
Ваша задача: вывести функцию потерь, градиент и гессиан для многоклассовой логистической регрессии. Реализовать матрично. На синтетическом примере продемонстрировать работу алгоритма, построить гиперплоскости, объяснить классификацию